# The Semipredicate Problem

The [semipredicate problem](https://en.wikipedia.org/wiki/Semipredicate_problem) is the problem of how to overcome ambiguity in distinguishing a real result from an indication that there is no result and/or that an error occurred.

A language-agnostic example of the semipredicate problem, and of solving it using *out-of-band signalling*, is having a primary output stream `stdout` and a secondary output / error output stream `stderr`.

An specific example from C is [ERR02-C. Avoid in-band error indicators](https://wiki.sei.cmu.edu/confluence/display/c/ERR02-C.+Avoid+in-band+error+indicators).

In [ ]:
'The horse said, "It\'s no good."'  # Quoting is an example of in-band signaling.

'The horse said, "It\'s no good."'

Suppose we have a dict `d` and a key `k`, and we want to do one thing if `k` is in `d` (using the associated value), and something else if it is not.

Provided we don't need to worry about the dict being mutated between checking and subscripting, we can use LBYL in a way that avoids the semipredicate problem arising:

```python
if k in d:
    v = d[k]
    ...  # Use the value v.
else:
    ...  # Deal with k not being mapped.
```

Often (but not always) a better approach is EAFP, which solves the semipredicate problem by using *returns* as the main source of information and *exceptions* as a secondary, out-of-band source of information:

```python
try:
    v = d[k]
except KeyError:
    ...  # Deal with k not being mapped.
else:
    ...  # Use the value v.
```

`dict` has a `get` method, which can be useful when neither of those patterns is convenient, but which brings back the semipredicate problem, since it uses in-band signaling.

In [ ]:
help(dict.get)

Help on method_descriptor:

get(self, key, default=None, /)
    Return the value for key if key is in the dictionary, else default.



In [ ]:
d = {7: "car", 3: "boat", 5: "airplane"}

In [ ]:
d.get(5)

'airplane'

In [ ]:
d.get(10)

In [ ]:
d.get(10,"ITS NOT HERE!")

'ITS NOT HERE!'

Suppose you are writing code in a function, and the situtaitons happens to be on where it is
much more convenient to use `dict.get` than to use `try`-`catch`. In *some* situations, you
will know for sure that a particular value will not be present. There are also situations
where there is no existing value that is totally safe as the default. It is nonetheless
possible to solve the semipredicate problem and use `dict.get`.

In [ ]:
# We need to make sure that o is not accessible to anything that might
# mutate d. So, in practice, we usually need o to be a local variable,
# thus this technique is, in practice, only useful in a function (but
# that's okay since most of our Python code is in a function anyway).
o = object()
if d.get(10, o) is o:
    print('Not here')    

Not here


In [ ]:
help(d.get)

Help on built-in function get:

get(key, default=None, /) method of builtins.dict instance
    Return the value for key if key is in the dictionary, else default.



In [ ]:
v = d.get(10, object())

In [ ]:
w = d.get(10, object())

In [ ]:
v == w

False

In [ ]:
type(object)

type

In [ ]:
d[10] = o

In [ ]:
def initial_practice(dictionary, key, value):
    try:
        value = dictionary[key]
    except KeyError:
        print("the key is not mapped") # Deal with key not being mapped.
    else:
        print(f'The value for {key} is {value}') # Use the value 

In [ ]:
d

{7: 'car', 3: 'boat', 5: 'airplane', 10: <object at 0x268d8f9cdf0>}

In [ ]:
initial_practice(d, 12, 'X wing')

the key is not mapped


In [ ]:
initial_practice(d, 7,'X wing')

The value for 7 is car


In [ ]:
def has_entry_eafp(dictionary, key, value):
    """
    Tell if a dictionary maps the given key to the given value.

    This implementation uses EAFP, catching KeyError.
    """
    try: 
        return dictionary[key] == value
    except KeyError:
        return False

In [ ]:
d

{7: 'car', 3: 'boat', 5: 'airplane', 10: <object at 0x268d8f9cdf0>}

In [ ]:
has_entry_eafp(d, 12, 'X wing')

False

In [ ]:
has_entry_eafp(d, 7, 'car')

True

In [ ]:
has_entry_eafp(d, 7, 'Tie Fighter')

False

In [ ]:
def has_entry_lbyl(dictionary, key, value):
    """
    Tell if a dictionary maps the given key to the given value.

    This implementation uses LBYL, checking with the "in" operator.
    """
    return (key in dictionary) and (dictionary[key] == value)

In [ ]:
has_entry_lbyl(d, 12, 'X wing')

False

In [ ]:
has_entry_lbyl(d, 7, 'car')

True

In [ ]:
has_entry_lbyl(d, 7, 'Tie')

False

In [ ]:
def has_entry_get(dictionary, key, value):
    """
    Tell if a dictionary maps the given key to the given value.

    This implementation uses the get method (in a safe way).
    """
    o = object()
    return dictionary.get(key, o) == value

In [ ]:
has_entry_get(d, 12, 'X wing')

False

In [ ]:
has_entry_get(d, 7, 'car')

True

In [ ]:
has_entry_get(d, 7, 'Tie')

False

In [ ]:
has_entry_get(d, 10, object())

False

In [ ]:
has_entry_get(d, 10, d[10])

True

In [ ]:
d[12] = 'Tie Fighter' 

In [ ]:
d

{7: 'car',
 3: 'boat',
 5: 'airplane',
 10: <object at 0x268d8f9cdf0>,
 12: 'Tie Fighter'}

## 2-argument forms of `next` and `iter`

Calling `next` with a second argument causes that argument to be returned instead of propagating `StopIteration` to the caller:

In [ ]:
help(next)

Help on built-in function next in module builtins:

next(...)
    next(iterator[, default])
    
    Return the next item from the iterator. If default is given and the iterator
    is exhausted, it is returned instead of raising StopIteration.



In [ ]:
it = iter([10, 20])

In [ ]:
next(it, 'Good bye!')

10

In [ ]:
next(it, 'Good bye!')

20

In [ ]:
next(it, 'Good bye!')

'Good bye!'

This is sometimes useful. **The usual considerations, regarding the semipredicate problem, that apply to `dict.get`, apply to calling `next` with two arguments.** Personally, I use two-argument `next` less often than I use `dict.get`, and I don&rsquo;t use either one regularly.

The two-argument `next` is a little bit unusual&mdash;and different from `dict.get`&mdash;because calling `next` with two arguments is very different from calling it with one argument, *no matter what second argument you pass*.

- Calling `dict.get` with no second argument is like calling it with `None`.

- Calling `next` with no second argument is not like calling it with any value as the second argument. When called with one argument, `next` always propagates `StopIteration` to the caller if the iterator has run out. But calling it with two arguments *never* does that&mdash;it returns the second argument instead.

What&rsquo;s going on here is that:

- The usual, one-argument form of `next` behaves in a manner analogous to *subscripting* a `dict`&mdash;when there is no value to be returned, calling `next` with one argument raises `StopIteration`, much as subscripting a `dict` raises `KeyError`.

- The less commonly used two-argument form of `next` behaves in a manner analogous to `dict.get` with an explicit second argument&mdash;when there is no value to be returned, calling `next` with two arguments returns the second argument, just as  calling `dict.get` returns its second argument.

`iter` can also be called with two arguments, which is totally different from calling it with one.

- Calling it with one argument returns an iterator to the iterable passed as the argument.

- Calling it with two arguments returns an iterator that repeatedly calls the first argument&mdash;which is expected to be a function or otherwise callable&mdash;and yields the values returned, until the value returned is equal to the second argument.

In [ ]:
help(iter)

Help on built-in function iter in module builtins:

iter(...)
    iter(iterable) -> iterator
    iter(callable, sentinel) -> iterator
    
    Get an iterator from an object.  In the first form, the argument must
    supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



The two-argument form of `iter` is rarely used, but it can be helpful in some situations.

In [ ]:
a = ['ham', 'spam', 'eggs', 'foo', 'bar', 'baz', 'foobar', 'quux']
it = iter(a.pop, 'eggs')
it

Notice the type&mdash;`callable_iterator`:

- The type of iterator `iter` returns when called with one argument is determined by what type of thing is being iterated, and the logic for doing so is supplied by that type.

- But when called with two arguments, `iter` synthesizes an iterator of type `callable_iterator`.

In [ ]:
next(it)

'quux'

In [ ]:
a  # a.pop has been called once so far.

['ham', 'spam', 'eggs', 'foo', 'bar', 'baz', 'foobar']

In [ ]:
list(it)

['foobar', 'baz', 'bar', 'foo']

In [ ]:
a

['ham', 'spam']

There is a conceptual connection between two-argument `next` and two-argument `iter`&mdash;they both involve end sentinels. **But they need not be used together and there is no special reason to use them together.** Whether one calls `iter` with one argument or two typically has no bearing on whether or not one calls `next` on the returned iterator with a second argument.

The two-argument form of `iter` also suffers from the semipredicate problem. Do you see how?